# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import gkey

gmaps.configure(api_key=gkey)
weather_file = os.path.join("..","output_data","cities.csv")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv(weather_file, encoding = "UTF-16")
weather_df.head(2)


,name,country,lat,lng,max_temp,temperature,humidity,cloudiness,wind_speed,date
0,Rikitea,PF,-23.1203,-134.9692,74.44,74.44,75,1,9.86,1624767295
1,Pārsābād,IR,39.6482,47.9174,84.18,84.18,42,59,3.38,1624767295


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
fig = gmaps.figure(map_type='TERRAIN', mouse_handling= "NONE",  zoom_level=1, center=(0, 0))
heatmap_layer = gmaps.heatmap_layer(locations = weather_df[["lat", "lng"]], weights = weather_df["humidity"])
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
good_temp = (weather_df["max_temp"] <= 75) & (weather_df["max_temp"] >= 65)
no_wind = weather_df["wind_speed"] <=10
clear_skies = weather_df["cloudiness"] <=10
not_a_swamp = weather_df["humidity"] <=50
vacation_df = weather_df[good_temp & no_wind & clear_skies & not_a_swamp]
len(vacation_df)

5

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:

def find_hotel(lat, lng, country, city):
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "key" : gkey,
        "input" : "Hotel",
        "inputtype" : "textquery",
        "locationbias" : f"circle:5000@{lat},{lng}",
        "fields" : "name"
    }
    r = requests.get(url, params)
    if (r.status_code == 200):
        c = r.json()['candidates']
        if len(c) > 0:
            hotel = c[0]['name']
            row = {"lat" : lat, "lng" : lng, "name" : hotel, "country" : country, "city" : city}
            return row

hotels = [find_hotel(lat, lng, country, city) for lat, lng, country, city in zip(vacation_df["lat"], vacation_df["lng"], vacation_df["country"], vacation_df["name"])]


In [53]:
hotels_df = pd.DataFrame(hotels)

In [50]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["lat", "lng"]]

In [52]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))